# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [6]:
# import libraries
import pandas as pd
import numpy as np

In [ ]:
# load messages dataset
messages = pd.read_csv("../data/disaster_messages.csv")
messages.head()

In [ ]:
# load categories dataset
categories = pd.read_csv("../data/disaster_categories.csv")
categories.head()

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [ ]:
# merge datasets
df = pd.merge(messages, categories, on="id")
df.head()

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.
- Update the total value of the items to be 0 or 1

In [ ]:
# create a dataframe of the 36 individual category columns
df_tmp = df['categories'].str.split(";", expand=True)
cols = [w.split("-")[0] for w in df_tmp.iloc[0, :].tolist()]
df_tmp.columns = cols
df_tmp = df_tmp.apply(lambda x: x.str.replace(".*?-([0-9]+)", r"\g<1>").astype(int)).clip(0, 1)
df = pd.concat([df.iloc[:, :-1], df_tmp], axis=1)

### 4. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# check number of duplicates
df_dub = df[df.duplicated(subset=None)]
print("Duplicates: {}".format(df_dub.shape[0]))
df_dub.head()

In [ ]:
# drop duplicates
df = df.drop_duplicates()

In [ ]:
# check number of duplicates
df[df.duplicated(subset=None)]

Check for items with a very low mean

In [ ]:
desc = df.describe()
desc[desc.columns[desc.loc['std', :] < 0.01]]

In [ ]:
# since child_alone holds no relevant information, remove:
df = df.drop('child_alone', axis=1)

### 5. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [53]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/disaster_data.db')
df.to_sql('texts', engine, index=False)

### 6. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [ ]:
df.head()